The Functions for Map Creation for Templates Django App Thesis

for MVP choosing one policy and applying it to entire city

In [111]:
import geojson
import geopandas
import pandas as pd
import numpy as np
import statistics as st
import matplotlib as plt
import folium
from folium.features import CustomIcon
import csv
from IPython.display import display, HTML
from pathlib import Path 
import os
from datetime import date, time, datetime
from dateutil import tz #for dealing with timezone
from geopy import distance
from collections import defaultdict

In [5]:
def load_data(s):
    #load data into notebook from sjoined shapefile
    geodataframe = geopandas.read_file(s)
    #print(gdf.head())
    # pass
    return geodataframe
# gdf = load_data('census_newvar_gold/census_newvar_gold.shp')
gdf = load_data('Out_CSV/census_longlat_gold/census_newvar_longlat_gold.shp')

In [39]:
gdf.columns
gdf.shape
columns_list = gdf.columns
len(columns_list)
print(columns_list[23])
for i in columns_list:
    pass
    #print(i)


license


In [43]:
#convert to dataframe because i know methods better 
df1 = pd.DataFrame(gdf)
#print(df1.loc[:,columns_list[23]])
#print(df1.loc[:,'listing_ur'])

In [137]:
#smaller dummy df
columns_df0 = ['id', 'has_liscen', 'rounded_re', 'price', 'name', 'host_id', 'host_name', 'census_tra', 'commercial', 'is_entire', 'effected_1', 'latitude', 'longitude','SEZ2011', 'P1', 'PF1', 'A47', 'A46', 'E3', 'E27', 'geometry']
df0 = df1.loc[:,columns_df0]
no_lisc_df0 = (df0.loc[df0['has_liscen'] == 0])
#print(no_lisc_df0.head())
shape_no_lisc = ((no_lisc_df0.shape))
lisc_df0 = (df0.loc[df0['has_liscen'] == 1])

print(lisc_df0['name'])
comm_no_lisc_df0 = (no_lisc_df0.loc[no_lisc_df0['commercial'] == 1])
nocomm_no_lisc_df0 = (no_lisc_df0.loc[no_lisc_df0['commercial'] == 0])
#print(comm_no_lisc_df0)
shape_comm_no_lisc_df0 = comm_no_lisc_df0.shape
#print(shape_comm_no_lisc_df0)

12                 Pinzochere 2DX Studio
13         Pinzochere 2SX by Santa Croce
14    Pinzochere Mansarda at SANTA CROCE
47    Alfani Suite by GALLERIA ACCADEMIA
48      Alfani Terrace 2b near the Duomo
49               Alfani 2b VIEW OF DUOMO
Name: name, dtype: object


In [105]:
#create stats of no liscenses
num_no_lisc = shape_no_lisc[0]
num_no_lisc_comm = shape_comm_no_lisc_df0[0]
num_no_lisc_noncomm = shape_no_lisc[0] - shape_comm_no_lisc_df0[0]
yearly_fee_nolisc = (num_no_lisc_noncomm * 30)
yearly_fee_nolis_comm = (num_no_lisc_comm * 60)
yearly_fee_tot = yearly_fee_nolisc + yearly_fee_nolis_comm
#print(yearly_fee_nolisc)
#no_lisc_df0.loc[:,'rounded_re'] #aggregate all, then tax 21% 
yearly_revenue_nolisc = no_lisc_df0['rounded_re'].agg('sum')
#print(yearly_revenue_nolisc)
yearly_increased_tax_rev_21 = yearly_revenue_nolisc * .21
#print(yearly_increased_tax_rev_21)
yearly_rev_nolisc_nocomm = nocomm_no_lisc_df0['rounded_re'].agg('sum')
yearly_rev_nolisc_comm = comm_no_lisc_df0['rounded_re'].agg('sum')
yearly_rev_tot =  (yearly_rev_nolisc_nocomm * .21) + (yearly_rev_nolisc_comm * .60)
print(yearly_rev_tot)
print(yearly_fee_tot)

28619.763
2430


In [158]:
#adding overlay census json data

census_json = 'Out_CSV/lighter_overlay_census_shape_attrib.geojson'
census_csv = 'Out_CSV/R09_fi.csv'
census_data = pd.read_csv(census_csv)
print(census_data.head())

m = folium.Map(
    location=[43.769, 11.255],
    tiles="cartodbpositron",
    zoom_start=12,
)
# folium.GeoJson(census_json, name="geojson").add_to(m)
# folium.LayerControl().add_to(m)

# folium.Choropleth(
#     geo_data=census_json,
#     name="choropleth",
#     data=census_data,
#     #census tract (SEZ2011), home renting (A46)
#     columns=["SEZ2011", "A46"],
#     key_on="feature.SEZ2011", #not sure wha tthe key on is supposed to be... was id from tutorial 
#     fill_color="YlGn",
#     fill_opacity=0.7,
#     line_opacity=0.2,
#     legend_name="Renter Count by Census Tract", #Rate (%)
# ).add_to(m)

# m

   CODREG  REGIONE  CODPRO PROVINCIA  CODCOM   COMUNE  PROCOM       SEZ2011  \
0       9  Toscana      48   Firenze      17  Firenze   48017  480170060001   
1       9  Toscana      48   Firenze      17  Firenze   48017  480170060002   
2       9  Toscana      48   Firenze      17  Firenze   48017  480170060003   
3       9  Toscana      48   Firenze      17  Firenze   48017  480170060004   
4       9  Toscana      48   Firenze      17  Firenze   48017  480170060005   

    NSEZ  ACE  ...  E22  E23  E24  E25  E26  E27  E28  E29  E30  E31  
0  60001   32  ...    8   10    9    1    0  126   20   19    0    1  
1  60002   32  ...    2    1    0    0    0    8    0    4    0    0  
2  60003   32  ...    0    0    1    0    0    8    0    1    0    0  
3  60004   32  ...    9    7    8    3    0  138   13   27    0    0  
4  60005   32  ...    0    0    0    1    0   14    0    0    1    0  

[5 rows x 152 columns]


In [159]:
#create map of liscenses
#lisc_df0
#trying ot add folium layer 
esri = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}'
map = folium.Map(location=[no_lisc_df0.latitude.mean(),no_lisc_df0.longitude.mean()], zoom_start=12, control_scale=True, tiles=esri, attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ, TomTom, Intermap, iPC, USGS, FAO, NPS, NRCAN, GeoBase, Kadaster NL, Ordnance Survey, Esri Japan, METI, Esri China (Hong Kong), and the GIS User Community') 
#too heavy...not showing up... 
#folium.GeoJson(census_json, name="geojson").add_to(map)
#adding option to toggel on and off
folium.LayerControl().add_to(map)
#CircleMarker adding markers
for index, location_info in no_lisc_df0.iterrows():
    folium.CircleMarker([location_info["latitude"],location_info["longitude"]], radius=2, color="crimson", fill=True, fill_color ="crimson",  popup=[location_info["name"],location_info["host_name"],location_info["commercial"], location_info["rounded_re"], location_info["SEZ2011"]], tooltip=location_info["price"]).add_to(map)
for index, location_info in lisc_df0.iterrows():
    folium.CircleMarker([location_info["latitude"],location_info["longitude"]], radius=2, color="blue", fill=True, fill_color ="blue",  popup=[location_info["name"],location_info["host_name"],location_info["commercial"], location_info["rounded_re"], location_info["SEZ2011"]], tooltip=location_info["price"]).add_to(map)
map.save('Out_Map/liscense_test.html')
map


In [40]:
#todo later
#create columns we want
#then make dataframe from that for gdf and df
columns_df2 = ['id', 'listing_ur',  'last_scrap', 'name', 'host_id', 'host_name', 'host_since', 'host_locat', 'host_total', neighbourh latitude longitude room_type accommodat bedrooms
'price',
availabili
availabi_1
availabi_2
availabi_3
number_of_
number_o_1
license
instant_bo
calculated
reviews_pe
days_rente
rounded_re
is_hotel
is_entire
many_listi
only_1_lis
only_2_lis
host_flore
has_liscen
is_instant
global_tot
dist_duomo
is_centro
is_gavinan
is_isolott
is_rifredi
is_campo
census_tra
listing_re
effected_b
effected_1
effected_2
commercial 
num_reside
num_househ
res_buildi
num_bedroo
num_hous_1
num_hous_2, SEZ2011, P1, PF1, A47, A46, E3, E27, 'geometry']
#df2 = df1.loc[:,]

0     https://www.airbnb.com/rooms/24469
1     https://www.airbnb.com/rooms/24470
2     https://www.airbnb.com/rooms/24471
3     https://www.airbnb.com/rooms/24472
4     https://www.airbnb.com/rooms/26738
5     https://www.airbnb.com/rooms/31840
6     https://www.airbnb.com/rooms/32120
7     https://www.airbnb.com/rooms/32180
8     https://www.airbnb.com/rooms/32395
9     https://www.airbnb.com/rooms/39115
10    https://www.airbnb.com/rooms/39165
11    https://www.airbnb.com/rooms/39822
12    https://www.airbnb.com/rooms/42997
13    https://www.airbnb.com/rooms/43365
14    https://www.airbnb.com/rooms/43369
15    https://www.airbnb.com/rooms/44781
16    https://www.airbnb.com/rooms/45606
17    https://www.airbnb.com/rooms/46520
18    https://www.airbnb.com/rooms/46523
19    https://www.airbnb.com/rooms/46524
20    https://www.airbnb.com/rooms/47498
21    https://www.airbnb.com/rooms/48372
22    https://www.airbnb.com/rooms/52543
23    https://www.airbnb.com/rooms/52545
24    https://ww

In [8]:
gdf.head()

,field_1,id,listing_ur,last_scrap,name,host_id,host_name,host_since,host_locat,host_total,...,E23,E24,E25,E26,E27,E28,E29,E30,E31,geometry
0,0,24469,https://www.airbnb.com/rooms/24469,2021-12-17,Fortezza/City Centre Modern Apt 2+2,99178,Benedetta And Lorenzo,2010-03-26,"Florence, Toscana, Italy",5,...,0,1,2,0,33,2,1,0,0,POINT (11.24392 43.78210)
1,1,24470,https://www.airbnb.com/rooms/24470,2021-12-17,Fortezza/City Centre Modern Apt 2+1,99178,Benedetta And Lorenzo,2010-03-26,"Florence, Toscana, Italy",5,...,4,4,2,1,79,3,6,2,0,POINT (11.24399 43.78202)
2,2,24471,https://www.airbnb.com/rooms/24471,2021-12-17,Fortezza/City Centre Modern Apt 4+2,99178,Benedetta And Lorenzo,2010-03-26,"Florence, Toscana, Italy",5,...,4,4,2,1,79,3,6,2,0,POINT (11.24399 43.78202)
3,3,24472,https://www.airbnb.com/rooms/24472,2021-12-17,Fortezza/City Centre Modern Apt 4+2,99178,Benedetta And Lorenzo,2010-03-26,"Florence, Toscana, Italy",5,...,4,4,2,1,79,3,6,2,0,POINT (11.24399 43.78202)
4,4,26738,https://www.airbnb.com/rooms/26738,2021-12-17,N4U Guest House Florence,113883,N4U Guest House,2010-04-26,"Florence, Toscana, Italy",2,...,1,0,1,0,13,0,2,0,0,POINT (11.25754 43.77017)


In [7]:
gdf.tail()

,field_1,id,listing_ur,last_scrap,name,host_id,host_name,host_since,host_locat,host_total,...,E23,E24,E25,E26,E27,E28,E29,E30,E31,geometry
46,46,77570,https://www.airbnb.com/rooms/77570,2021-12-17,Duomo Flat Amazing view in Florence,416144,Massimiliano,2011-03-01,"Florence, Toscana, Italy",2,...,2,2,4,0,72,3,11,2,0,POINT (11.25914 43.77272)
47,47,80692,https://www.airbnb.com/rooms/80692,2021-12-17,Alfani Suite by GALLERIA ACCADEMIA,4156491,Apartments Florence,2012-11-15,"Florence, Tuscany, Italy",189,...,0,3,0,0,22,0,4,2,0,POINT (11.26009 43.77556)
48,48,80693,https://www.airbnb.com/rooms/80693,2021-12-17,Alfani Terrace 2b near the Duomo,4156491,Apartments Florence,2012-11-15,"Florence, Tuscany, Italy",189,...,0,3,0,0,22,0,4,2,0,POINT (11.26009 43.77556)
49,49,80694,https://www.airbnb.com/rooms/80694,2021-12-17,Alfani 2b VIEW OF DUOMO,4156491,Apartments Florence,2012-11-15,"Florence, Tuscany, Italy",189,...,0,3,0,0,22,0,4,2,0,POINT (11.26009 43.77556)
50,50,81606,https://www.airbnb.com/rooms/81606,2021-12-17,MAGDA: - close to the DUOMO-SPECIAL APRIL-MAY,434231,Magda,2011-03-10,"Viareggio, Tuscany, Italy",11,...,2,15,3,0,153,1,12,11,7,POINT (11.25881 43.77360)


In [6]:
gdf.explore()

In [5]:
def original_census_map(s):
    #create chloropleth map
    pass
    return chl_map
#original_census_map(s)

In [4]:
def original_airbnb_map(s):
    #create bubble map
    pass
    return bub_map
#original_airbnb_map(s)

In [ ]:
def stats(s):
    #calculate stats for given area before any regulation
    #metrics policy 1: tax, #listings (total, total per census, liscenses, total number of bedrooms, total per census block) 
    #metrics for others commercial, whos host has more than 2 listings, more than 90 days)  
    pass
    return fi_stats
#stats(s)

In [10]:
def updated_census_map(s):
    #create updated chloropleth map after clicking a policy x
    pass
    return updated_chl_map
#updated_census_map(s)

In [ ]:
def updated_airbnb_map(s):
    #create updated bubble map after clicking certain policy x
    pass
    return updated_bub_map
#updated_bub_map(s)

In [ ]:
def updated_stats(s):
    #after user clicks policy x update stats
     #x number new liscenses, fee generated, tax %30 for all previously unliscensed based on revenue 
    pass
    return new_fi_stats
#policy_interaction_stats(s)

## Here we can create the python that will eventually go into each of the above functions

In [ ]:
#chloropleth maps with census data by census tract:
#showing by color shade the density of airbnb listing count
#pop up folium tool tip saying for that census tract x% of bedrooms are listed on airbnb, if 10 bedrooms total, and 5 bedrooms on airbnb then 50%
#showing by color shade (different hue) homeowner versus renter 
#pop up folium tool tip saying for that census tract x% are renters vs homeowners


In [ ]:
#colored bubble maps with airbnb joined data by census tract:
#color of bubble can indicate if likely commercial, in or out of florence, more than 90days, has a liscnese or not, has more than 2 listings


In [ ]:
#graduated bubble maps with airbnb joined data by census tract:
#size can be linked to # of listings, 1:100, # of days rented a year, # of revenue (hue color for this too)